# DNBLab Jupyter Notebook Tutorial

## SRU-Abfragen erklärt - Tutorial für Einsteiger\*innen (JupyterLite-Version) 

Dieses Tutorial beschreibt, wie Sie in der browserbasierten JupyterLite-Umgebung und der Programmiersprache Python die SRU-Schnittstelle der DNB abfragen und mit den erhaltenen Antworten arbeiten können. Der Aufbau der Abfragen wird anhand von Beispielen erklärt und stützt sich auf die Dokumentation der SRU-Schnittstelle unter https://www.dnb.de/sru.

Das Tutorial ist wie folgt aufgebaut: 

* [1. Einrichten der Arbeitsumgebung](#Teil1) 
* [2. Abfragen verschiedener Datensätze der DNB](#Teil2)  
* [3. Aufbau einer gezielten Suche](#Teil3) 

<font color="red"><strong>Hinweis:</strong></font> JupyterLite ist eine schnelle, "leichtgewichtige" und sehr ressourcenschonende Coding-Umgebung. Dadurch stehen in JupyterLite nicht alle Funktionalitäten zur Verfügung und der Code des Tutorials wurde im Vergleich zur Jupyter Notebook-Version entsprechend angepasst. 

## 1. Einrichten der Arbeitsumgebung

Um die Arbeitsumgebung für die folgenden Schritte einzurichten, werden zunächst die benötigten Python-Biblitoheken importiert: "urllib", "pyodide" und "js" für die Abfragen an die SRU-Schnittstelle in der JupyterLite-Umgebung, "ElementTree" über "lxml" (als ET) und "BeautifulSoup", um die XML-Antworten der Schnittstelle besser verarbeiten zu können sowie "unicodedata" zur Verarbeitung der Zeichencodierung:

In [2]:
import urllib.parse
from pyodide.http import open_url, pyfetch
from js import fetch
from bs4 import BeautifulSoup as soup
import unicodedata
from lxml import etree as ET

Die SRU-Schnittstelle der DNB ist unter der Basis-URL https://services.dnb.de/sru erreichbar. Ein Aufruf dieser Adresse im Browser zeigt den aktuellen Status sowie die Version der Schnittstelle an. 

## 2. Abfrage verschiedener Datensätze der DNB 

Die DNB bietet Ihre Daten über drei verschiedene "Kataloge" an, die entsprechend für eine Abfrage ausgewählt werden müssen. Dies geschieht über eine Erweiterung der o.g. Basis-URL. Zur Verfügung stehen folgende Kataloge: 

* Katalog der Deutschen Nationalbibliothek (DNB) - hierin befinden sich die Titeldaten
* Katalog des Deutschen Musikarchivs (DMA) - Datensätze des Deutschen Musikarchivs
* Katalog der Gemeinsamen Normdatei (GND) - hierin befinden sich die Normdaten

Die Erweiterungen für die URL sind folgende: 

* DNB: https://services.dnb.de/sru/dnb
* DMA: https://services.dnb.de/sru/dnb.dma
* GND: https://services.dnb.de/sru/authorities

Werden die jeweiligen Bereiche ohne weitere Spezifikationen abgefragt, senden sie eine Selbstbeschreibung im XML-Standardformat https://services.dnb.de/sru/dnb?operation=explain&version=1.1 zurück. 


In [3]:
#URL der SRU-Schnittstelle der DNB: 
base_url = "https://services.dnb.de/sru/dnb"

#Anfrage - speichern der Antwort in die Variable "basic_request" und Überführung des Inhalts in die Variable "response":
basic_request =  await fetch(base_url)
content = await basic_request.text()

Mit Hilfe der Bibliothek "BeautifulSoup" kann die Antwort direkt in XML umgewandelt werden. 

<font color="red"><strong>Hinweis:</strong></font>  Um das Tutorial übersichtlich zu halten, wird die Ausgabe der Antwort im folgenden auf die ersten 500 Zeichen gekürzt - die eigentliche Antwort ist länger und kann durch einfaches Löschen der Einschränkung "[0:500]" in der "print"-Zeile komplett angezeigt werden. Natürlich können auch andere Bereiche zur Anzeige gewählt werden. 

In [4]:
#Umwandeln in XML und Ausgabe der ersten 500 Zeichen: 
response = soup(content, features="xml")
print(response.prettify()[0:500])

<?xml version="1.0" encoding="utf-8"?>
<explainResponse xmlns="http://www.loc.gov/zing/srw/">
 <version>
  1.1
 </version>
 <record>
  <recordSchema>
   http://explain.z3950.org/dtd/2.0/
  </recordSchema>
  <recordPacking>
   xml
  </recordPacking>
  <recordData>
   <ns:explain id="Deutsche Nationalbibliothek" xmlns:ns="http://explain.z3950.org/dtd/2.0/">
    <ns:serverInfo protocol="sru" version="1.1">
     <ns:host>
      services.dnb.de
     </ns:host>
     <ns:port>
      443
     </ns:port>


Für eine Suchanfrage an die Daten der DNB wird nun über die URL der Katalog definiert. Mit Hilfe der Variable *parameter* werden dann alle weiteren benötigten Parameter übergeben. 

Besonders relevant sind dabei die beiden Parameter 'query' : 'Klimawandel', sowie 'recordSchema' : 'MARC21-xml'. Statt "Klimawandel" kann hier jeder beliebige Suchbegriff eingetragen werden - auch Suchanfragen, die aus mehreren Wörtern bestehen, können mittels boolscher Operatoren übergeben werden. Die genaue Syntax wird unter https://www.dnb.de/sru beschrieben. Statt "MARC21-xml" stehen außerdem noch die Ausgabeformate "oai_dc" oder "RDFxml" als Ausgabeformate zur Verfügung (siehe weiter unten).   


In [5]:
base_url = "https://services.dnb.de/sru/dnb"
params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'query': "Klimawandel"
         }
    
r = await fetch(base_url + "?" + urllib.parse.urlencode(params))  
r_text = await r.text()

response = soup(r_text, features="xml")
print(response.prettify()[0:500])

<?xml version="1.0" encoding="utf-8"?>
<searchRetrieveResponse xmlns="http://www.loc.gov/zing/srw/">
 <version>
  1.1
 </version>
 <numberOfRecords>
  9979
 </numberOfRecords>
 <records>
  <record>
   <recordSchema>
    MARC21-xml
   </recordSchema>
   <recordPacking>
    xml
   </recordPacking>
   <recordData>
    <record type="Bibliographic" xmlns="http://www.loc.gov/MARC21/slim">
     <leader>
      00000nam a22000008c 4500
     </leader>
     <controlfield tag="001">
      1147699615
     </


Zu beachten ist, dass die Suche nach einem Stichwort über den 'query'-Befehl eine allgemeine Suche über alle Daten des ausgewählten Katalogs darstellt. Die Suche ist nicht auf Titel oder ähnliches beschränkt, sondern durchsucht die Datensätze im Gesamtindex der Katalogdaten. Zum Beispiel kann auf diese Weise auch nach Namen gesucht werden, die dann allerdings nicht nur unter Autor*innen, sondern auch in Titeln gefunden werden.

## 3. Aufbau einer gezielten Suche 

Zur Eingrenzung der Suche auf bestimmte Angaben wie Titel oder Autor*in können unter anderem folgende zusätzliche Parameter genutzt werden:

* tit= Suche im Titeleintrag
* atr= Suche nach Verfasser*in (Person oder Organisation)
* per= Suche nach Personen (in allen relevanten Feldern)
* sw = Suche nach Schlagworten
* jhr = Suche nach Erscheinungszeitraum
...

Eine detaillierte Übersicht der verschiedenen Abfragemöglichkeiten gibt es unter https://www.dnb.de/expertensuche. Dabei können die unterschiedlichen Parameter auch beliebig in der Suchanfrage kombiniert werden.

Für die Ausgabe der Ergebnisse kann außerdem zwischen drei Formaten gewählt werden, indem der entsprechende Code hinter "recordSchema" geändert wird:

* MARC21-xml (XML-Variante von MARC 21)
* oai_dc (DNB Casual - Auswahl von Dublin-Core-Elementen - nur für Titeldaten!)
* RDFxml (RDF - Linked Data Service)

Eine Suchanfrage nach Titeln, die das Suchwort "Klimawandel" enthalten, im Jahr 2005 erschienen sind und im Format DNB Casual "oai_dc" angefragt wird, sieht wie folgt aus:

In [7]:
#Parameter, die der Anfrage übergeben werden: 
parameter = {'version' : '1.1' , 'operation' : 'searchRetrieve' , 'query' : 'tit=Klimawandel and jhr=2005',
             'recordSchema' : 'oai_dc', 'maximumRecords': '100'} 

r = await fetch(base_url + "?" + urllib.parse.urlencode(parameter))  
r_text = await r.text()

#Parsen der Antwort "r" als XML in die neue Variable "response":
response = soup(r_text, features="xml")

#Ausgabe der ersten 1000 Zeichen: 
print(response.prettify()[0:750])

<?xml version="1.0" encoding="utf-8"?>
<searchRetrieveResponse xmlns="http://www.loc.gov/zing/srw/">
 <version>
  1.1
 </version>
 <numberOfRecords>
  29
 </numberOfRecords>
 <records>
  <record>
   <recordSchema>
    oai_dc
   </recordSchema>
   <recordPacking>
    xml
   </recordPacking>
   <recordData>
    <dc xmlns="http://www.openarchives.org/OAI/2.0/oai_dc/" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:dnb="http://d-nb.de/standards/dnbterms" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
     <dc:title>
      Anpassung an den Klimawandel : Gründe, Folgen, Handlungsoptionen / Bundesministerium für Wirtschaftliche Zusammenarbeit und Entwicklung ; Gtz, Deutsche Gesellschaft für Technische Zusammenarbeit (GTZ) GmbH
     </dc:


Der erste XML-Block enthält die Gesamtzahl der gefundenen Ergebnisse:  

```
<numberOfRecords>
   ZAHL
</numberOfRecords>
```

Wenn man diese Information nicht im XML suchen möchte, kann auch der Code so angepasst werden, dass der entsprechende Abschnitt mithilfe des Zusatzes ".find('numberOfRecords')" gesucht und als Attribut ".text" an die Variable *number* angehängt wird.


number = response.find('numberOfRecords')
print(number.text, 'Ergebnisse')

#Einfache Ausgabe:
#print(number)

Da die einzelzen Treffer bzw. Datensätze jeweils durch "record"-Tags gekennzeichnet sind, werden diese nun gesucht und in der Variable records zwischengespeichert. Im Anschluss wird die Länge der Variable ausgeben,um sie mit der Angabe unter "numberOfRecords" zu vergleichen.

<font color="red"><strong>Hinweis:</strong></font>  Die SRU-Schnittstelle gibt immer erstmal nur die ersten 100 Treffer zurück, d.h. auch bei größeren Treffermengen wird maximal die Länge 100 angezeigt. Wie man größere Treffermengen sammeln kann, folgt weiter unten im Tutorial.

In [8]:
records = response.find_all('record')
print(len(records), 'Ergebnisse')

29 Ergebnisse


<font color="red"><strong>Hinweis:</strong></font> Da die Datensätze bei einer geänderten Ausgabe in 'recordSchema' : 'MARC21-xml' anstatt des voreingestellten 'recordSchema' : 'oai_dc' unterschiedlich verschachtelt sein können, muss neben der Auswahl des Katalogs hinter der Basis-URL "/dnb" oder "/authorities" bei der oben stehenden Suche zusätzlich der Typ des Datensatzes angegeben werden:


MAR21-xml Titeldaten (/dnb): <bold> `records = response.find_all('record', {'type':'Bibliographic'})` </bold> <br>
MAR21-xml Normdaten: (/authorities) <bold> `records = response.find_all('record', {'type':'Authority'})` </bold>


Die Ergebnisse werden als Liste gespeichert und stehen dabei jeweils an einem Platz innerhalb der Listenvariable. Bei 9 Ergebnissen werden in der Liste die Plätze 0-8 belegt, was bei der Adressierung bedacht werden muss.

Der 3. Eintrag wird entsprechend über den Listenplatz Nummer 2 aufgerufen:

In [10]:
print(records[2])

<record><recordSchema>oai_dc</recordSchema><recordPacking>xml</recordPacking><recordData><dc xmlns="http://www.openarchives.org/OAI/2.0/oai_dc/" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:dnb="http://d-nb.de/standards/dnbterms" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
<dc:title>Die EU im Einsatz gegen den Klimawandel : der EU-Emissionshandel - ein offenes System, das weltweit Innovationen fördert / [Europäische Kommission]</dc:title>
<dc:creator>Europäische Kommission</dc:creator>
<dc:publisher>[Luxemburg] : [Amt für Amtliche Veröff. der Europ. Gemeinschaften]</dc:publisher>
<dc:date>2005</dc:date>
<dc:language>ger</dc:language>
<dc:identifier xmlns:tel="http://krait.kb.nl/coop/tel/handbook/telterms.html" xsi:type="tel:ISBN">92-894-9187-6 geh.</dc:identifier>
<dc:identifier xsi:type="dnb:IDN">992017882</dc:identifier>
<dc:subject>360 Soziale Probleme, Sozialdienste, Versicherungen</dc:subject>
<dc:subject>330 Wirtschaft</dc:subject>
<dc:format>20 S.</dc:format>
</d

Die bisherigen Schritte können wie folgt in einer Funktion gefolgt von der Abfrage zusammengefasst werden:

In [11]:
#Funktion
async def dnb_sru_short(query):
    
    base_url = "https://services.dnb.de/sru/dnb"
    parameter = {'recordSchema' : 'oai_dc',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
          'query': query
         }
    
    r = await fetch(base_url + "?" + urllib.parse.urlencode(parameter))  
    content = await r.text()
    xml = soup(content, features="xml")
    records = xml.find_all('record')
    
    return records
    


In [12]:
#Formulierung der Abfrage: 
myquery = await dnb_sru_short('tit=Klimawandel and jhr=2005') #Aufruf der Funktion 'sru-dnb' mit der Abfrage 'tit=Klimawandel...'
print(len(myquery), "Ergebnisse")

29 Ergebnisse


Die SRU-Schnittstelle gibt zunächst maximal 100 Treffer aus. Wenn es sich um mehr Ergebnisse handelt, können weitere Treffer mithilfe einer "Schleife" in 100er Schritten geholt und zwischengespeichert werden. Das so gesammelte Ergebnis wird ausgegeben, sobald die Ergebnismenge aus "numberOfRecords" erreicht wurde:

In [13]:
#Funktion
async def dnb_sru(query):
    
    base_url = "https://services.dnb.de/sru/dnb"
    parameter = {'recordSchema' : 'oai_dc',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
          'query': query
         }
    
    r = await fetch(base_url + "?" + urllib.parse.urlencode(parameter))  
    content = await r.text()
    xml = soup(content, features="xml")
    number = int(xml.find('numberOfRecords').text)
    records = xml.find_all('record')
    
    if number <= 101:       # wurden maximal 100 Treffer gefunden? Wenn ja, erfolgt direkt die Rückgabe.
        return records
    
    else:                   # wurden mehr als 100 Treffer gefunden, wird hier die Schleife gestartet.
        num_results = 100
        i = 101
        
        while num_results == 100:
            
            parameter.update({'startRecord': i}) 
            r = await fetch(base_url + "?" + urllib.parse.urlencode(parameter))  
            content = await r.text()
            xml = soup(content, features="xml")
            new_records = xml.find_all('record')
            records+=new_records
            i+=100
            num_results = len(new_records)
            
        return records
    

Eine Abfrage für das Titelstichwort "Klimawandel" kombiniert mit dem Jahr 2019 ergibt folgende Treffermenge:

In [14]:
myquery = await dnb_sru('tit=Klimawandel and jhr=2019')
print(len(myquery), "Ergebnisse")

305 Ergebnisse


Die so abgerufenen Treffer können entweder direkt weiterverarbeitet oder lokal zwischengespeichert werden. 
Im Folgenden werden die Ergebnisse in eine XML-Datei geschrieben, die heruntergeladen werden kann: 

In [16]:
with open('sru_abfrage_klimawandel.xml', 'w', encoding="utf-8") as f:
    f.write(str(myquery))